# 경쟁사 고객 리뷰 분석으로 소비자 인식 조사 
영양제 브랜드 자사와 경쟁사의 구매 후기를 분석해서 소비자들의 인식을 조사해봅니다. 

## 데이터 설명
데이터는 다음의 링크에서 다운받으실 수 있습니다.


---

* [센트룸 구매후기](https://drive.google.com/file/d/1eeTHELYDR0UW9CK7yODGhcUGylTfpLv4/view?usp=sharing)
*  [세노비스 카페 데이터](https://drive.google.com/file/d/1798xQJmU2rIlqiLyoUcIhelGV9szjLUu/view?usp=sharing)
* [센트룸 카페 데이터](https://drive.google.com/file/d/1RFYWzhrMGongGVMIEvs1gaMOi-PmAfuz/view?usp=sharing)

---


# 텍스트 마이닝을 위한 전처리 
## KoNLP를 이용한 형태소 분석 
- KoNLPy가 제공하는 형태소분석기 중 하나인 Kkma를 사용합니다.
- 자세한 내용은 http://konlpy.org/ko/v0.4.3/morph/ 참조

In [0]:
# KoNLPy 라이브러리 설치. 최초 1회만 실행
!pip install konlpy

# 형태소 분석기
- 한나눔 http://semanticweb.kaist.ac.kr/hannanum/index.html
- 트위터 https://github.com/twitter/twitter-korean-text
- 꼬꼬마 http://kkma.snu.ac.kr/documents/

## 형태소분석기의 성능차이
![대체 텍스트](http://konlpy.org/ko/v0.4.2/_images/time.png)
* 출처: http://konlpy.org/ko/v0.4.2/morph/

In [3]:
from konlpy.tag import Hannanum
from konlpy.tag import Twitter
from konlpy.tag import Kkma
hannanum=Hannanum()
twitter=Twitter()
kkma = Kkma()

/home/ubuntu/.local/lib/python3.6/site-packages/konlpy/tag/_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


### 꼬꼬마 형태소 분석기 
- 문장을 형태소 단위로 분리하고 품사를 태깅합니다
- 품사태그는 일반명사(NNG), 고유명사(NNP), 동사(VV), 형용사(VA) 등이 있습니다
- http://kkma.snu.ac.kr/documents/index.jsp?doc=postag 형태소 리스트를 확인

In [4]:
print(kkma.sentences(u'아버지가 방에 들어가셨다. 아버지 가방에 들어가셨다. 아버지가 방 안에 있는 가방에 들어가셨다.'))

['아버지가 방에 들어가셨다.', '아버지 가방에 들어가셨다.', '아버지가 방 안에 있는 가방에 들어가셨다.']


In [5]:
print(kkma.pos(u'아버지가 방에 들어가셨다. 아버지 가방에 들어가셨다. 아버지가 방 안에 있는 가방에 들어가셨다.'))

[('아버지', 'NNG'), ('가', 'JKS'), ('방', 'NNG'), ('에', 'JKM'), ('들어가', 'VV'), ('시', 'EPH'), ('었', 'EPT'), ('다', 'EFN'), ('.', 'SF'), ('아버지', 'NNG'), ('가방', 'NNG'), ('에', 'JKM'), ('들어가', 'VV'), ('시', 'EPH'), ('었', 'EPT'), ('다', 'EFN'), ('.', 'SF'), ('아버지', 'NNG'), ('가', 'JKS'), ('방', 'NNG'), ('안', 'NNG'), ('에', 'JKM'), ('있', 'VV'), ('는', 'ETD'), ('가방', 'NNG'), ('에', 'JKM'), ('들어가', 'VV'), ('시', 'EPH'), ('었', 'EPT'), ('다', 'EFN'), ('.', 'SF')]


### 한나눔 형태소 분석기

In [6]:
print(hannanum.pos(u'아버지가 방에 들어가셨다. 아버지 가방에 들어가셨다. 아버지가 방 안에 있는 가방에 들어가셨다.'))

[('아버지', 'N'), ('가', 'J'), ('방', 'N'), ('에', 'J'), ('들', 'P'), ('어', 'E'), ('가', 'P'), ('셨다', 'E'), ('.', 'S'), ('아버지', 'N'), ('가방', 'N'), ('에', 'J'), ('들', 'P'), ('어', 'E'), ('가', 'P'), ('셨다', 'E'), ('.', 'S'), ('아버지', 'N'), ('가', 'J'), ('방', 'N'), ('안', 'N'), ('에', 'J'), ('있', 'P'), ('는', 'E'), ('가방', 'N'), ('에', 'J'), ('들', 'P'), ('어', 'E'), ('가', 'P'), ('셨다', 'E'), ('.', 'S')]


### 트위터 형태소 분석기

In [7]:
print(twitter.pos('아버지가 방에 들어가셨다. 아버지 가방에 들어가셨다. 아버지가 방 안에 있는 가방에 들어가셨다.'))

[('아버지', 'Noun'), ('가', 'Josa'), ('방', 'Noun'), ('에', 'Josa'), ('들어가셨다', 'Verb'), ('.', 'Punctuation'), ('아버지', 'Noun'), ('가방', 'Noun'), ('에', 'Josa'), ('들어가셨다', 'Verb'), ('.', 'Punctuation'), ('아버지', 'Noun'), ('가', 'Josa'), ('방', 'Noun'), ('안', 'Noun'), ('에', 'Josa'), ('있는', 'Adjective'), ('가방', 'Noun'), ('에', 'Josa'), ('들어가셨다', 'Verb'), ('.', 'Punctuation')]


# 텍스트 마이닝 분석 및 시각화
* 센트룸 데이터를 먼저 분석해봅니다. 

### 데이터 탐색 및 전처리

In [0]:
# 데이터를 불러옵니다.
from google.colab import files
uploaded = files.upload()

In [0]:
# 형태소분석 - kkma 명사 
line_list = []
f = open("centrum_review.txt", encoding="utf-8")
for line in f:
    line = kkma.nouns(line)
    line_list.append(line)
f.close()

print("- 불러온 문서 :", len(line_list), "문장")

In [0]:
line_list[10]

In [0]:
word_frequency = {}
noun_list = []
#불용어 리스트를 여기에 추가합니다.
stop_list = ["배송", "만족"]
line_number = 0
for line in line_list[:]:
    line_number += 1
    print(str(line_number) + "/" + str(len(line_list)), end="\r")
    noun = []
    for word in line:
        if word.split("/")[0] not in stop_list and len(word.split("/")[0]) > 1:  
            noun.append(word.split("/")[0])
            if word not in word_frequency.keys():
                word_frequency[word] = 1
            else:
                word_frequency[word] += 1
    noun_list.extend(noun)

In [0]:
#단어별 출현빈도를 출력합니다.
word_count = []
for n, freq in word_frequency.items():
    word_count.append([n, freq])
word_count.sort(key=lambda elem: elem[1], reverse=True)
for n, freq in word_count[:10]:
    print(n + "\t" + str(freq))

In [0]:
#추출한 명사 리스트를 활용해 명사만으로 이루어진 문서를 생성합니다.
noun_doc = ' '.join(noun_list)
noun_doc = noun_doc.strip()

* 서체 다운로드 
- 시각화에서 서체 변경만으로도 완성도를 높일 수 있습니다. 
- [다음의 링크](https://drive.google.com/file/d/1ULZI7vTml7mVVrM3fbo4AW8xhyETDETx/view?usp=sharing)에서 나눔스퀘어 서체를 다운로드 받아주세요. 
- 참고: https://hangeul.naver.com/2017/nanum

In [0]:
# 워드클라우드 시각화
from wordcloud import WordCloud, ImageColorGenerator
import matplotlib.pyplot as plt

#폰트업로드
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [0]:
#워드클라우드 파라미터 설정
font_path="NanumSquareB.otf"  #폰트
background_color="white"      #배경색
margin=3                     #모서리 여백 넓이
min_font_size=7              #최소 글자 크기
max_font_size=150             #최대 글자 크기
width=500                     #이미지 가로 크기
height=500                    #이미지 세로 크기
wc = WordCloud(font_path=font_path, background_color=background_color, margin=margin, \
               min_font_size=min_font_size, max_font_size=max_font_size, width=width, height=height)
wc.generate(noun_doc)

plt.figure(figsize=(10, 10))
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.show()

In [0]:
# 데이터를 불러옵니다.
from google.colab import files
uploaded = files.upload()

In [0]:

# 형태소분석 - kkma 명사 
line_list = []
f = open("centrum2.txt", encoding="utf-8")
for line in f:
    line = kkma.nouns(line)
    line_list.append(line)
f.close()

print("- 불러온 문서 :", len(line_list), "문장")

In [0]:
word_frequency = {}
noun_list = []
#불용어 리스트를 여기에 추가합니다.
stop_list = ["배송", "만족", "카페", "카페규정", "확인", "주수", "센트"]
line_number = 0
for line in line_list[:]:
    line_number += 1
    print(str(line_number) + "/" + str(len(line_list)), end="\r")
    noun = []
    for word in line:
        if word.split("/")[0] not in stop_list and len(word.split("/")[0]) > 1:  
            noun.append(word.split("/")[0])
            if word not in word_frequency.keys():
                word_frequency[word] = 1
            else:
                word_frequency[word] += 1
    noun_list.extend(noun)

In [0]:
#단어별 출현빈도를 출력합니다.
word_count = []
for n, freq in word_frequency.items():
    word_count.append([n, freq])
word_count.sort(key=lambda elem: elem[1], reverse=True)
for n, freq in word_count[:10]:
    print(n + "\t" + str(freq))

In [0]:
#추출한 명사 리스트를 활용해 명사만으로 이루어진 문서를 생성합니다.
noun_doc = ' '.join(noun_list)
noun_doc = noun_doc.strip()

In [0]:
#워드클라우드 파라미터 설정
font_path="NanumSquareB.otf"  #폰트
background_color="white"      #배경색
margin=3                     #모서리 여백 넓이
min_font_size=7              #최소 글자 크기
max_font_size=150             #최대 글자 크기
width=500                     #이미지 가로 크기
height=500                    #이미지 세로 크기
wc = WordCloud(font_path=font_path, background_color=background_color, margin=margin, \
               min_font_size=min_font_size, max_font_size=max_font_size, width=width, height=height)
wc.generate(noun_doc)

plt.figure(figsize=(10, 10))
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.show()

### LDA 토픽 모델링

In [0]:
import gensim
from gensim import corpora
import logging
logging.basicConfig(level=logging.DEBUG)
topic = 5
keyword = 10
texts = []
resultList = []
stop_list = ["배송", "만족", "카페", "카페규정", "확인", "주수", "센트"]
for line in line_list:
    words = line
    if words != [""]:
        tokens = [word for word in words if (len(word.split("/")[0]) > 1 and word.split("/")[0] not in stop_list)]
        texts.append(tokens)
dictionary = corpora.Dictionary(texts)    
corpus = [dictionary.doc2bow(text) for text in texts]

ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=topic, id2word = dictionary, passes=10)
for num in range(topic):
    resultList.append(ldamodel.show_topic(num, keyword))

In [0]:
resultList